# Loading images and convolve it before CNN

In [ ]:
import numpy as np
import cv2
from skimage.exposure import rescale_intensity

In [ ]:
height, width = 256, 256
cover_path = 'BOSSbase_1.01-256/cover/0/'
stego_path = 'BOSSbase_1.01-256/stego/0/'

In [ ]:
def convolve(image, kernel):
    i_width, i_height = image.shape[0], image.shape[1]
    k_width, k_height = kernel.shape[0], kernel.shape[1]
    pad = k_width // 2
    output = np.zeros((i_width - 2*pad, i_height - 2*pad), dtype="float32")

    for y in range(pad, i_height - pad):
        for x in range(pad, i_width - pad):
            weighted_pixel_sum = 0
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            for ky in range(0, k_height):
                for kx in range(0, k_width):
                    weighted_pixel_sum += roi[ky, kx] * kernel[ky, kx]
            output[y - pad, x - pad] = weighted_pixel_sum

    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
 
    # return the output image
    return output

In [ ]:
kernel = np.array((
    [-1,  2,  -2,  2, -1],
    [ 2, -6,   8, -6,  2],
    [-2,  8, -12,  8, -2],
    [ 2, -6,   8, -6,  2],
    [-1,  2,  -2,  2, -1]), dtype="float")
kernel = np.divide(kernel, 12)

In [ ]:
cover_file_data = np.empty([height, width])

stego_file_data = np.empty([height, width])

for file_number in range(1, 1001):
    cover_image = cv2.imread(cover_path + str(file_number) + '.pgm', cv2.IMREAD_GRAYSCALE)
    cover_file_data = np.vstack((cover_file_data, cover_image))
    
    stego_image = cv2.imread(stego_path + str(file_number) + '.pgm', cv2.IMREAD_GRAYSCALE)
    stego_file_data = np.vstack((stego_file_data, stego_image))

cover_file_data = np.delete(cover_file_data.reshape(1001, height, width), 0,0)

stego_file_data = np.delete(stego_file_data.reshape(1001, height, width), 0,0)

In [ ]:
height = height - 2 * (kernel.shape[0] // 2)
width = width - 2 * (kernel.shape[0] // 2)

In [ ]:
cover_convolved_data = np.empty([1000, height, width])
for i in range(1000):
    cover_convolved_data[i] = convolve(cover_file_data[i], kernel)
    
stego_convolved_data = np.empty([1000, height, width])
for i in range(1000):
    stego_convolved_data[i] = convolve(stego_file_data[i], kernel)

# Preparing data

In [ ]:
#zeros for cover; ones for stego
cover_train_data = cover_convolved_data.reshape(1000,-1)
cover_train_data = np.c_[np.zeros([1000,1]), cover_train_data]

stego_train_data = stego_convolved_data.reshape(1000,-1)
stego_train_data = np.c_[np.ones([1000,1]), stego_train_data]

In [ ]:
cover_train_data.shape

In [ ]:
stego_train_data.shape

In [ ]:
cover_X_train = cover_train_data[:,1:]
cover_Y_train = cover_train_data[:,0]

stego_X_train = stego_train_data[:,1:]
stego_Y_train = stego_train_data[:,0]

In [ ]:
X_train = np.concatenate((cover_X_train, stego_X_train), axis = 0)
Y_train = np.concatenate((cover_Y_train, stego_Y_train), axis = 0)

X_train = X_train.reshape(X_train.shape[0], height, width, 1)

In [ ]:
X_train.shape, Y_train.shape

# GNCNN Model

In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, AveragePooling2D, Activation
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from livelossplot import PlotLossesKeras

In [ ]:
batch_size = 200
# seed = 3243
num_classes = 2

In [ ]:
X_train = X_train / 255
Y_train = to_categorical(Y_train)

In [ ]:
def gaussian_activation(x):
    sigma = 1
    return K.exp(-(x*x)/(sigma*sigma))

get_custom_objects().update({'gaussian_activation': Activation(gaussian_activation)})

In [ ]:
model = Sequential()
model.add(Conv2D(16, 5, activation='gaussian_activation', padding='same', input_shape=(height, width, 1)))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='same'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='same'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='same'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 5, activation='gaussian_activation', padding='same'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile('adam', 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train, validation_split=0.1, epochs=100, batch_size=batch_size,
                 callbacks=[
#                      EarlyStopping(patience=3),
                     PlotLossesKeras()])